In [1]:
!python --version

Python 3.11.12


## Importing Libraries and API Setup


In [2]:
import json
import requests
from bs4 import BeautifulSoup
import re
from transformers import pipeline
from tqdm import tqdm
from google.colab import userdata

api_key = userdata.get('NewsAPI')
base_url = "https://newsapi.org/v2/everything"

# Possible models we could use
# facebook/bart-large-cnn
# facebook/bart-large-xsum
# google/pegasus-xsum
# google/pegasus-multi_news
# google/bigbird-pegasus-large-arxiv
# google/pegasus-cnn_dailymail

## Fetching Article Text and Getting Apple Finance Articles

In essence, this code retrieves news articles about Apple's finances from the News API, extracts relevant information, fetches the full content of each article, and stores all this data in a structured way.

In [3]:
def fetch_full_article_text(url):
    """
    The function fetches the full text content of an article from a given URL by extracting paragraphs
    or main article tags using BeautifulSoup in Python.

    :param url: The `fetch_full_article_text` function is designed to fetch the full text content of an
    article from a given URL. It uses the `requests` library to make an HTTP request to the URL and
    `BeautifulSoup` to parse the HTML content of the page
    :return: The function `fetch_full_article_text(url)` returns the full text content of the article
    found at the specified URL. If there is an error during the process of fetching and parsing the
    webpage, it will return `None`.
    """
    try:
        page = requests.get(url, timeout=5)
        soup = BeautifulSoup(page.content, "html.parser")

        # Extract paragraphs or main article tag
        paragraphs = soup.find_all("p")
        full_text = " ".join([p.get_text() for p in paragraphs])

        return full_text.strip()
    except Exception as e:
        print(f"Error fetching full text from {url}: {e}")
        return None

def get_apple_finance_articles(api_key, base_url, num_articles=5):
    """
    The function `get_apple_finance_articles` retrieves a specified number of finance articles related
    to Apple using an API key and base URL.

    :param api_key: The `api_key` parameter in the `get_apple_finance_articles` function is used to
    authenticate your access to the API. It is a unique identifier that allows you to make requests to
    the API and retrieve data related to Apple finance articles. You need to obtain this API key from
    the API
    :param base_url: The `base_url` parameter in the `get_apple_finance_articles` function is typically
    the URL endpoint of the API you are making a request to in order to fetch Apple finance articles. It
    should be a string representing the base URL of the API service you are using to retrieve the
    articles
    :param num_articles: The `num_articles` parameter in the `get_apple_finance_articles` function
    specifies the number of articles you want to retrieve related to Apple finance. By default, it is
    set to 5, but you can provide a different number when calling the function to fetch a specific
    number of articles, defaults to 5 (optional)
    :return: The function `get_apple_finance_articles` returns a list of dictionaries, where each
    dictionary represents information about an article related to "Apple finance". The dictionary
    contains keys such as "source", "author", "publishedAt", "title", "description", "url", "content",
    and "content_len" for each article. If there is an error during the process of fetching articles,
    it will return an empty list.
    """
    # Keywords and parameters for the API request
    query = "Apple finance"
    query_params = {
        "q": query,
        "apiKey": api_key,
        "language": "en",
        "sortBy": "relevancy",  # Sort results by relevancy
        "pageSize": num_articles  # Limit the number of results
    }

    try:
        response = requests.get(base_url, params=query_params)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        articles = []
        if data["status"] == "ok":
            for article in data["articles"]:
                source = article.get("source")
                author = article.get("author")
                publishedAt = article.get("publishedAt")
                title = article.get("title")
                description = article.get("description")
                url = article.get("url")
                content = fetch_full_article_text(url)
                content_len = len(content) if content else 0

                if title and description and url: # Ensure article data exists
                    article_info = {
                        "source": source,
                        "author": author,
                        "publishedAt": publishedAt,
                        "title": title,
                        "description": description,
                        "url": url,
                        "content": content,
                        "content_len": content_len
                    }
                    articles.append(article_info)

        return articles
    except requests.exceptions.RequestException as e:
        print(f"Error fetching articles: {e}")
        return []

# Get the `num_articles` most relevant articles
articles = get_apple_finance_articles(api_key, base_url, num_articles=100)

In [4]:
len(articles)

91

In [5]:
# Export articles to a JSON file
try:
    with open("articles.json", "w") as json_file:
        json.dump(articles, json_file, indent=4)  # Use indent for pretty printing
    print("Articles exported to articles.json")
except Exception as e:
    print(f"Error exporting articles to JSON: {e}")


Articles exported to articles.json


## Data Cleaning and Preprocessing

This part of the code focuses on cleaning and preparing the text of the articles for summarization. It involves two main functions: `clean_text` and `preprocess_articles`. It also defines two variables that dictate the length of tokens or text processed.

In [6]:
MAX_TOKENS = 1024  # model-specific, for BART/PEGASUS
MAX_CHARS = 3000   # rough estimate if tokenization is too heavy

def clean_text(text):
    """
    The `clean_text` function removes HTML tags, normalizes whitespace, removes references, comments,
    and URLs from the input text.

    :param text: The `clean_text` function takes a text input and performs several cleaning operations
    on it. It removes HTML tags, normalizes whitespace, removes references enclosed in square brackets,
    removes comments enclosed in parentheses, and removes URLs. Finally, it strips any leading or
    trailing whitespace before returning the cleaned text
    :return: The `clean_text` function returns the cleaned version of the input text after removing HTML
    tags, normalizing whitespace, removing references, comments, and URLs. The cleaned text is then
    stripped of leading and trailing whitespaces before being returned.
    """
    if not text:
        return ""
    text = re.sub(r"<.*?>", "", text)  # remove HTML tags
    text = re.sub(r"\s+", " ", text) # normalize whitespace
    text = re.sub(r"\[[^]]*\]", "", text)  # remove [references]
    text = re.sub(r"\([^)]*\)", "", text)  # remove (comments)
    text = re.sub(r"http\S+", "", text)    # remove URLs
    return text.strip()

def preprocess_articles(news_items):
    """
    The `preprocess_articles` function processes news articles by cleaning the content and filtering out
    unusable articles based on their length.

    :param news_items: The `preprocess_articles` function takes a list of news items as input. Each news
    item is a dictionary containing information about an article, such as its title, content, URL,
    source, author, publication date, description, and content length
    :return: The `preprocess_articles` function processes a list of news items and returns a new list of
    dictionaries containing information about each article. Each dictionary in the returned list
    includes the following keys:
    """
    processed = []
    for article in news_items:
        clean_content = clean_text(article.get("content", ""))
        if not clean_content or len(clean_content) < 200:
            clean_content = clean_text(article.get("description", ""))

        if not clean_content or len(clean_content) < 200:
            print("Article: \"" + article.get("title", "") + "\" is unsuable!")
            print("Content: \"" + article.get("content", "") + "\"")
            print("-" * 20)
            continue  # skip unusable articles

        # if len(clean_content) > MAX_CHARS:
        #     clean_content = clean_content[:MAX_CHARS]  # truncate

        processed.append({
            "title": article.get("title", ""),
            "url": article.get("url", ""),
            "source": article.get("source", {}).get("name", ""),
            "author": article.get("author", ""),
            "publishedAt": article.get("publishedAt", ""),
            "description": article.get("description", ""),
            "content": article.get("content", ""),
            "content_len": article.get("content_len", ""),
            "clean_content": clean_content,
            "clean_content_len": len(clean_content),
        })
    return processed

In [7]:
preprocessed_articles = preprocess_articles(articles)

Article: "Tesla must do these 3 on its earnings call, or else" is unsuable!
Content: ""
--------------------
Article: "Getting a tax refund from the CRA? Here's how you can get the most out of it" is unsuable!
Content: ""
--------------------
Article: "Bitcoin inches closer to flipping NVIDIA in stunning market surge" is unsuable!
Content: "Please enable JS and disable any ad blocker"
--------------------
Article: "Working from home is turning into 'return to the office or lose your bonus'" is unsuable!
Content: ""
--------------------
Article: "London rents rise to record high for 14th consecutive month" is unsuable!
Content: ""
--------------------
Article: "Stock market today: S&P 500, Nasdaq plunge, Dow drops 1,400 points as Trump's tariffs shock markets" is unsuable!
Content: ""
--------------------
Article: "Stock market today: Dow surges 600 points, S&P 500 has best week since 2023 to cap wild week of tariff-fueled chaos" is unsuable!
Content: ""
--------------------
Article: "I

In [8]:
len(preprocessed_articles)

83

In [9]:
preprocessed_articles[0]

{'title': 'Apple May Face Criminal Charges for Allegedly Lying to a Federal Judge',
 'url': 'https://www.wired.com/story/antitrust-judge-asks-doj-prosecute-apple/',
 'source': 'Wired',
 'author': 'Paresh Dave',
 'publishedAt': '2025-05-01T00:20:26Z',
 'description': 'A US judge says Apple deliberately chose not to comply with an order requiring it to loosen App Store rules—and then tried to cover up its disobedience.',
 'content': 'Apple “willfully chose not to comply” with a court order to loosen its app store restrictions—and one of its executives lied under oath about the company’s plans, a federal judge wrote on Wednesday. Judge Yvonne Gonzalez Rogers has referred the situation to the US Attorney’s Office in San Francisco “to investigate whether criminal contempt proceedings are appropriate.” In 2021, Gonzalez Rogers presided over a lawsuit brought by Fortnite developer Epic Games over the iPhone maker’s allegedly anticompetitive practices that hampered the ability of developers to

##Model Selection and Initialization

This code snippet is responsible for selecting the models that will be used for text summarization and initializing them for use.

In [ ]:
# List of models we want to use
model_names = [
    "facebook/bart-large-cnn",
    # "facebook/bart-large-xsum",
    "google/pegasus-xsum",
    "google/pegasus-multi_news",
    # "google/bigbird-pegasus-large-arxiv",
    "google/pegasus-cnn_dailymail"
]

# Create summarizers for each model
summarizers = {model_name: pipeline("summarization", model=model_name, device=-1) for model_name in model_names}

In [11]:
def summarize_text(summarizer, text, max_input_length=1024, min_summary_length_ratio=0.1, max_summary_length_ratio=0.3):
    """
    This function takes a text input, truncates it to a specified length, dynamically calculates summary
    lengths based on input length, and then uses a summarizer model to generate a summary within the
    calculated length limits.

    :param summarizer: The `summarizer` parameter in the `summarize_text` function is expected to be a
    function or model that can generate a summary of a given input text. This function/model should take
    the input text, along with optional parameters like `max_length` and `min_length`, and return
    :param text: The `summarize_text` function takes a `summarizer` function, `text` to be summarized,
    and optional parameters for input length, minimum summary length ratio, and maximum summary length
    ratio
    :param max_input_length: The `max_input_length` parameter in the `summarize_text` function specifies
    the maximum number of characters allowed in the input text that will be passed to the summarizer for
    summarization. If the input text exceeds this length, it will be truncated to fit within this limit
    before being summarized, defaults to 1024 (optional)
    :param min_summary_length_ratio: The `min_summary_length_ratio` parameter in the `summarize_text`
    function represents the minimum ratio of the input text length that the summary should aim to be. It
    dynamically calculates the minimum length of the summary based on this ratio. The function ensures
    that the summary length is at least 20
    :param max_summary_length_ratio: The `max_summary_length_ratio` parameter in the `summarize_text`
    function determines the maximum length of the summary relative to the length of the input text. It
    is used to dynamically calculate the maximum length of the summary based on a ratio of the input
    text length. The function ensures that the
    :return: The function `summarize_text` returns the summarized text based on the input text using the
    provided summarizer model. If there is no content to summarize (empty text), it returns '[No content
    to summarize]'. If there is an error during the summarization process, it returns '[Error during
    summarization]'. Otherwise, it returns the generated summary of the input text within the specified
    length constraints based on the provided summarizer model.
    """
    if not text or text.strip() == '':
        return '[No content to summarize]'

    # Truncate to model's max input length
    input_text = text.strip()[:max_input_length]
    input_len = len(input_text.split())  # count words

    # Dynamically calculate summary lengths
    min_length = max(20, int(input_len * min_summary_length_ratio))  # at least 20 words
    max_length = max(min_length + 10, int(input_len * max_summary_length_ratio))  # ensure max > min

    # Clip to model max length limits
    max_length = min(max_length, 200)  # you can adjust cap
    min_length = min(min_length, max_length - 10) if max_length > 10 else min_length

    try:
        summary = summarizer(input_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    except Exception as e:
        print(f"⚠️ Error summarizing: {e}")
        summary = '[Error during summarization]'

    return summary

summarized_articles = []

for idx, article in enumerate(tqdm(preprocessed_articles, desc="Summarizing articles")):
    content = article.get('content', '') or ''
    title = article.get('title', '') or ''
    description = article.get('description', '') or ''

    input_text = content.strip()
    if not input_text:
        input_text = f"{title}. {description}".strip()

    if len(input_text) == 0:
        print(f"⚠️ Article {idx} has no content/title/description.")

    summarized_article = article.copy()  # copy original article

    # Summarize with each model
    for model_name, summarizer in summarizers.items():
        print(f"🔍 Summarizing article {idx} with {model_name}")
        summary = summarize_text(summarizer, input_text)
        summarized_article[f"summary_{model_name}"] = summary

    summarized_articles.append(summarized_article)

Summarizing articles:   0%|          | 0/83 [00:00<?, ?it/s]

🔍 Summarizing article 0 with facebook/bart-large-cnn
🔍 Summarizing article 0 with google/pegasus-xsum
🔍 Summarizing article 0 with google/pegasus-multi_news
🔍 Summarizing article 0 with google/pegasus-cnn_dailymail


Summarizing articles:   1%|          | 1/83 [01:36<2:11:20, 96.10s/it]

🔍 Summarizing article 1 with facebook/bart-large-cnn
🔍 Summarizing article 1 with google/pegasus-xsum
🔍 Summarizing article 1 with google/pegasus-multi_news
🔍 Summarizing article 1 with google/pegasus-cnn_dailymail


Summarizing articles:   2%|▏         | 2/83 [03:11<2:08:48, 95.41s/it]

...

Summarizing articles:  98%|█████████▊| 81/83 [1:44:14<02:29, 74.81s/it]

🔍 Summarizing article 81 with facebook/bart-large-cnn
🔍 Summarizing article 81 with google/pegasus-xsum
🔍 Summarizing article 81 with google/pegasus-multi_news
🔍 Summarizing article 81 with google/pegasus-cnn_dailymail


Summarizing articles:  99%|█████████▉| 82/83 [1:45:47<01:20, 80.14s/it]

🔍 Summarizing article 82 with facebook/bart-large-cnn
🔍 Summarizing article 82 with google/pegasus-xsum
🔍 Summarizing article 82 with google/pegasus-multi_news
🔍 Summarizing article 82 with google/pegasus-cnn_dailymail


Summarizing articles: 100%|██████████| 83/83 [1:47:04<00:00, 77.41s/it]


## Exporting Data

This function's main job is to take the summarized articles and save them into two separate files: one for training a machine learning model and one for testing it. These files are in JSONL format, which means each line is a JSON object representing a single article and its summary.

### JSONL vs JSON for Machine Learning Datasets
While both JSON and JSONL can store data, JSONL (JSON Lines) is often a better choice for large datasets used in machine learning.
So now the code is account for scalability.

Here's why:

1. Efficiency and Scalability:

  * JSONL: Each article and its summary are stored on a separate line. This makes it much faster and easier to process large datasets because you can read and parse each line independently.
  * JSON: The entire dataset is stored as a single, large JSON object. This can become very slow and memory-intensive to work with when dealing with many articles.

2. Ease of Streaming:

  * JSONL: Its line-by-line structure makes it ideal for streaming data. You can start processing articles immediately as they're read, without having to load the entire dataset into memory first. This is crucial for very large datasets that might not fit in memory.
  * JSON: Requires loading the entire file into memory before processing, which can be problematic for large datasets.

3. Flexibility:

  * JSONL: Each line can have a slightly different structure if needed, as long as it's valid JSON. This can be useful when dealing with data that has variations or optional fields.
  * JSON: Requires a strict, consistent structure for the entire dataset.

In [12]:
def export_articles_to_jsonl(summarized_articles, summary_key, train_file="train_dataset.jsonl", test_file="test_dataset.jsonl", train_size=50):
    """
    This function exports summarized articles to JSONL format, splitting them into training and testing
    datasets based on a specified size.

    :param summarized_articles: The `summarized_articles` parameter is a list of dictionaries where each
    dictionary represents an article with keys like "clean_content" and the specified `summary_key`
    containing the article content and its summary respectively. The function `export_articles_to_jsonl`
    takes this list of articles and exports them to
    :param summary_key: The `summary_key` parameter in the `export_articles_to_jsonl` function is used
    to specify the key in each article dictionary that contains the summary information. This key is
    used to extract the summary from the article dictionary and include it in the exported JSONL files
    :param train_file: The `train_file` parameter is the file path where the training dataset will be
    exported in JSONL format. By default, it is set to "train_dataset.jsonl", defaults to
    train_dataset.jsonl (optional)
    :param test_file: The `test_file` parameter in the `export_articles_to_jsonl` function is the file
    path where the test dataset will be exported in JSONL format. This file will contain articles from
    the `summarized_articles` list starting from the index `train_size` onwards, defaults to
    test_dataset.jsonl (optional)
    :param train_size: The `train_size` parameter in the `export_articles_to_jsonl` function specifies
    the number of articles that will be included in the training dataset. In this case, the function
    will export the first `train_size` articles to the `train_dataset.jsonl` file, and the remaining
    articles will, defaults to 50 (optional)
    """
    try:
        with open(train_file, "w", encoding="utf-8") as f:
            for article in summarized_articles[:train_size]:
                json.dump({"text": article["clean_content"], "summary": article[summary_key]}, f)
                f.write("\n")
    except Exception as e:
        print(f"Error exporting articles to {train_file}: {e}")

    try:
        with open(test_file, "w", encoding="utf-8") as f:
            for article in summarized_articles[train_size:]:
                json.dump({"text": article["clean_content"], "summary": article[summary_key]}, f)
                f.write("\n")
    except Exception as e:
        print(f"Error exporting articles to {test_file}: {e}")


In [13]:
export_articles_to_jsonl(summarized_articles, "summary_facebook/bart-large-cnn", train_file="train_bart-large-cnn.jsonl", test_file="test_bart-large-cnn.jsonl")
export_articles_to_jsonl(summarized_articles, "summary_google/pegasus-xsum", train_file="train_pegasus-xsum.jsonl", test_file="test_pegasus-xsum.jsonl")
export_articles_to_jsonl(summarized_articles, "summary_google/pegasus-multi_news", train_file="train_pegasus-multi_news.jsonl", test_file="test_pegasus-multi_news.jsonl")
export_articles_to_jsonl(summarized_articles, "summary_google/pegasus-cnn_dailymail", train_file="train_pegasus-cnn_dailymail.jsonl", test_file="test_pegasus-cnn_dailymail.jsonl")

In [14]:
# Export articles to JSON files for training (fine-tune) process
try:
    with open("train_dataset.jsonl", "w", encoding="utf-8") as f:
        for article in summarized_articles[0:50]:
            json.dump({"text": article["clean_content"], "summary": article["summary_facebook/bart-large-cnn"]}, f)
            f.write("\n")
except Exception as e:
    print(f"Error exporting articles to JSONL: {e}")

try:
    with open("test_dataset.jsonl", "w", encoding="utf-8") as f:
        for article in summarized_articles[50:]:
            json.dump({"text": article["clean_content"], "summary": article["summary_facebook/bart-large-cnn"]}, f)
            f.write("\n")
except Exception as e:
    print(f"Error exporting articles to JSONL: {e}")

In [15]:
# Export articles to JSON files for prediction process
try:
    with open("train_summarized_articles.jsonl", "w") as json_file:
        json.dump(summarized_articles[0:50], json_file, indent=4)
    print("Articles exported to train_summarized_articles.jsonl")
except Exception as e:
    print(f"Error exporting articles to JSON: {e}")

try:
    with open("test_summarized_articles.jsonl", "w") as json_file:
        json.dump(summarized_articles[50:], json_file, indent=4)
    print("Articles exported to test_summarized_articles.jsonl")
except Exception as e:
    print(f"Error exporting articles to JSON: {e}")

Articles exported to train_summarized_articles.jsonl
Articles exported to test_summarized_articles.jsonl
